# How to use logging


Estimagic can keep a persistent log of the parameter and criterion values tried out by an optimizer in a sqlite database. 

The sqlite database is also used to exchange data between the optimization and the dashboard.

## Turn logging on or off

To enable logging, it suffices to provide a path to an sqlite database when calling ``maximize`` or ``minimize``. The database does not have to exist, estimagic will generate it for you. 

In [ ]:
import numpy as np

import estimagic as em

In [ ]:
def sphere(params):
    return params @ params

In [ ]:
res = em.minimize(
    criterion=sphere,
    params=np.arange(5),
    algorithm="scipy_lbfgsb",
    logging="my_log.db",
)

## Make logging faster

By default, we use a very safe mode of sqlite that makes it almost impossible to corrupt the database. Even if your computer is suddenly shut down or unplugged. 

However, this makes writing logs rather slow, which becomes notable when the criterion function is very fast. 

In that case, you can enable ``"fast_logging"``, which is still quite safe!

In [ ]:
res = em.minimize(
    criterion=sphere,
    params=np.arange(5),
    algorithm="scipy_lbfgsb",
    logging="my_log.db",
    log_options={"fast_logging": True},
)

## Handling existing tables

By default, we only append to databases and do not overwrite data in them. You have a few options to change this:

In [ ]:
res = em.minimize(
    criterion=sphere,
    params=np.arange(5),
    algorithm="scipy_lbfgsb",
    logging="my_log.db",
    log_options={
        "if_database_exists": "replace",  # one of "raise", "replace", "extend",
        "if_table_exists": "replace",  # one of "raise", "replace", "extend"
    },
)

## Reading the log

In [ ]:
reader = em.OptimizeLogReader("my_log.db")

Read the start params

In [ ]:
reader.read_start_params()

Read a specific iteration (use -1 for the last)

In [ ]:
reader.read_iteration(-1)

Read the full history

In [ ]:
reader.read_history().keys()

## Plot the history from a log

In [ ]:
fig = em.criterion_plot("my_log.db")
fig.show(renderer="png")

In [ ]:
fig = em.params_plot("my_log.db", selector=lambda x: x[1:3])
fig.show(renderer="png")